In [1]:
from sklearn.datasets import load_boston
from sklearn.metrics import explained_variance_score
import numpy as np

x = load_boston().data
y = load_boston().target

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=29)

x_train.shape

(379, 13)

In [3]:
def polynom(m, deg):
    n_rows, n_cols = m.shape[0], m.shape[1]
    nm = np.zeros((m.shape[0],n_cols*deg))
    for d in range(deg):
        for c in range(n_cols):
            for r in range(n_rows):
                nm[r][n_cols*d+c] = m[r][c]**(d+1)
    return nm

In [4]:
# scaling
def scaling(X):
    for c in range(X.shape[1]):
        X[:,c] = [(i - np.mean(X[:,c])) / (max(X[:,c]) - min(X[:,c])) for i in X[:,c]]
    return X

# normalize
def normal(X):
    return X / X.max(axis=0)

In [5]:
def predict(X, theta):
    return np.c_[np.ones(len(X)), normal(X)] @ theta

In [6]:
def normalEq(X, y):
    X = np.c_[np.ones(len(X)), normal(X)]
    return (np.linalg.inv(X.T @ X) @ X.T @ y).T

theta = normalEq(x_train, y_train)

print(theta)

preds = predict(x_test, theta)

print(f'\nNorm Eq EVS: {explained_variance_score(preds, y_test)}')

[ 35.98189827  -6.88140442   5.31800698   1.01840583   3.43338847
 -12.89840309  31.08338357  -0.25493095 -16.56781587   7.73387839
  -9.19866233 -21.23376392   4.39090559 -20.92958659]

Norm Eq EVS: 0.717097886997718


In [71]:
class Ridge:
    def __init__(self, a):
        self.a = a
    
    def fit(self, X, y):
        X = np.c_[np.ones(len(X)), normal(X)]
        self.theta = (np.linalg.inv(X.T @ X + self.a * np.identity(1)) @ X.T @ y)
        
    def predict(self, X):
        return np.c_[np.ones(len(X)), normal(X)] @ self.theta

m = Ridge(10)

m.fit(x_train, y_train)

preds = m.predict(x_test)
        
# theta = RidgeEq(x_train, y_train, 0.1)

# print(theta)

# preds = predict(x_test, theta)

print(f'\nNorm Eq EVS: {explained_variance_score(preds, y_test)}')


Norm Eq EVS: 0.7181512345558685


$X^{'}X=\begin{bmatrix} 7 & 38.5\\  38.5& 218.75 \end{bmatrix}$

In [146]:
def BGD(X, y, n_iter, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X @ theta.T) - y.T).T @ nX
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = BGD(polynom(x_train, 2), y_train, 1000, 0.001)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\nBGD EVS: {explained_variance_score(preds, y_test)}')

[[ 21.48175763  -5.21052653   0.49326403  -2.29180405   1.8618664
    1.24390193  12.11210382  -3.66071495  -9.05726146   6.44094765
   -3.72503253  -1.54039884   6.20513171 -26.47508868  -1.76233164
    4.44617089   4.0114199    1.64128015  -7.56912652  20.10511533
    3.00061925  -2.81398974   0.1993134   -4.05174292  -9.73012564
   -1.04832018  10.66914994]]

BGD EVS: 0.7404059544112933


In [8]:
import time

start = time.clock()

def learning_schedule(t):
    return 5 / (t + 50)

def SGD(X, y, n_epoch, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(1, n_epoch):
        for n in range(m):
            for c in range(X.shape[1]):
                r = np.random.randint(0, m-1)
                xi, yi = X[r:r+1], y.T[r:r+1]
                gradients = ((xi @ theta.T) - yi.T).T @ xi[:,c]
#                 eta = eta/epoch
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T

theta = SGD(polynom(x_train, 2), y_train, 50, 0.01)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\nSGD EVS: {explained_variance_score(preds, y_test)}')

print(time.clock() - start)

[[  2.24643147e+00  -3.29898102e+00   2.32214303e+00   2.92197089e+00
    1.33970949e+00  -4.20696380e+00   2.53506742e+01  -5.99173023e+00
   -5.57913969e+00   7.06379388e+00  -3.37520686e-03   5.32311689e+00
    5.73467437e+00  -1.54072693e+01  -2.66497405e+00   4.23739371e+00
    1.26171947e-01   1.22708776e+00  -5.23986648e+00   2.05447971e+01
    7.34671249e+00  -3.14806413e+00   3.20087895e+00  -1.29556703e+01
   -9.94858007e+00   5.21667687e-02   1.88069379e+00]]

SGD EVS: 0.6928754526820999
3.7555449999999997


In [33]:
import time

start = time.clock()

def MBGD(X, y, n_epoch, batch_size, eta):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    m = X.shape[0]
    for epoch in range(n_epoch):
        shuffi = np.random.permutation(m)
        xs, ys = X[shuffi], y.T[shuffi]
        for i in range(0, m, batch_size):
            for c in range(X.shape[1]):
                xi, yi = X[i:i+batch_size], y.T[i:i+batch_size]
                gradients = ((xi @ theta.T) - yi).T @ xi[:,c]
                theta[:,c] = theta[:,c] - (gradients * eta)
    return theta.T
        
theta = MBGD(polynom(x_train, 2), y_train, 500, 64, 0.01)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\n MBGD EVS: {explained_variance_score(preds, y_test)}')

print(time.clock() - start)

[[ 36.90890086 -16.83969786  -3.86163523  -9.35246913   5.72236384
    2.64044615  -3.10944942  -5.93332168 -17.31346032  10.91323228
   -5.20743124   5.26539832  12.55465042 -55.50831314   8.34740545
    7.34089644  10.82089486  -2.39204345 -10.31394287  23.92606408
    5.94830668   4.89327161  -1.55528663  -3.25194018 -13.86446479
   -7.52477399  38.84850446]]

 MBGD EVS: 0.7548311531412629
0.5438309999999973


In [170]:
def Ridge(X, y, n_iter, eta, alpha):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    best = theta
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = (((X @ theta.T) - y.T).T @ nX) + 2 * alpha * theta[:,c]
            theta[:,c] = nT - eta * gradients
        b = explained_variance_score(X @ best.T, y.T)
        c = explained_variance_score(X @ theta.T, y.T)
        if c > b:
            best = theta
    return (best.T, theta.T)

theta, l = Ridge(polynom(x_train, 2), y_train, 10000, 0.01, 0.1)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

predsl = predict(polynom(x_test, 2), l)

print(f'\nRidge Best EVS: {explained_variance_score(preds, y_test)}')
print(f'\nRidge Last EVS: {explained_variance_score(predsl, y_test)}')

[[ 30.41618575 -12.55667436  -1.3485853   -1.73591433   2.87456095
   14.42178196   9.72087653  -2.44594127 -14.62455899  10.61420675
   -5.93581858  -9.48040865   5.094256   -47.91393294   4.20024488
    6.10805002   3.50970284   0.47668857 -17.18621574  15.2040884
    2.62020808   3.7896093   -2.64599867  -2.36364123  -5.63750186
   -0.83820171  31.1940496 ]]

Ridge Best EVS: 0.7409223943640557

Ridge Last EVS: 0.7409223943640557


In [161]:
def LASSO(X, y, n_iter, eta, alpha):
    X = np.c_[np.ones(len(X)), normal(X)]
    y = np.array([y.T])
    theta = np.random.rand(1, X.shape[1])
    for i in range(n_iter):
        for c in range(X.shape[1]):
            nX = X[:,c]
            nT = theta[:,c]
            gradients = ((X @ theta.T) - y.T).T @ nX + alpha * np.sign(theta[:,c])
            theta[:,c] = nT - eta * gradients
    return theta.T

theta = LASSO(polynom(x_train, 2), y_train, 1000, 0.001, 1)

print(theta.T)

preds = predict(polynom(x_test, 2), theta)

print(f'\nLASSO EVS: {explained_variance_score(preds, y_test)}')

[[  2.08884746e+01  -5.14840774e+00   4.75692581e-01  -1.70879486e+00
    1.78731340e+00   1.20804742e-03   1.22803641e+01  -2.77124626e+00
   -8.77196132e+00   6.16238114e+00  -3.66365665e+00  -1.70155230e+00
    5.79840315e+00  -2.57371678e+01  -1.17227021e+00   4.38311653e+00
    3.30821586e+00   1.71327155e+00  -6.42050946e+00   2.02263920e+01
    2.26221454e+00  -2.68663558e+00  -7.93849276e-03  -3.51672115e+00
   -9.41384602e+00  -5.94744492e-01   9.91499675e+00]]

LASSO EVS: 0.736123868444756


In [118]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()
polynomial_regression = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
polynomial_regression.fit(x_train, y_train)
preds = polynomial_regression.predict(x_test)

In [119]:
m = Ridge(alpha = 10)

m.fit(polynom(x_train, 2), y_train)

preds = m.predict(polynom(x_test, 2))

print('weights: ')
print(m.coef_)
print('Intercept: ')
print(m.intercept_)

print(f'\n Scikit learn ridge reg EVS: {explained_variance_score(y_test, preds)}')

weights: 
[ -4.45795003e-01  -4.20281709e-02  -1.13638279e-01   1.26681724e+00
  -1.43233686e+00  -2.18378528e+00  -3.29550428e-02  -1.37782981e+00
   5.43332742e-01  -3.16984967e-02  -1.81378244e+00   2.67667275e-02
  -1.80841526e+00   4.67544909e-03   7.78662251e-04   3.97695328e-03
   1.26681724e+00  -1.98542113e+00   4.07584210e-01   3.40675265e-04
   4.48095854e-02  -7.28036088e-03   2.27704613e-05   3.08977838e-02
  -3.93823629e-05   3.58743805e-02]
Intercept: 
66.143471914

 Scikit learn ridge reg EVS: 0.7704624730353917
